In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word-duplicates/new_df.csv


In [2]:
new_df= pd.read_csv('/kaggle/input/word-duplicates/new_df.csv')

In [3]:
new_df.shape


(49606, 18)

In [4]:
new_df.head()

,Unnamed: 0,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_share,cwc_min,cwc_max,last_word_eq,first_word_eq,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,398782,best marketing automation tool small mid size ...,best marketing automation tool small mid sized...,1,53,54,8,8,7,0.44,0.874989,0.874989,1.0,1.0,99,98,99,99
1,115086,poor want invest,quite poor want rich,0,16,20,3,4,2,0.29,0.666644,0.499988,0.0,0.0,61,73,50,72
2,327711,india live abroad met guy france party want date,e thapar university thapar university institut...,0,48,90,9,11,0,0.00,0.000000,0.000000,0.0,0.0,26,40,30,30
3,367788,many people u hate southern state,boyfriend doesnt feel guilty hurt cried tellin...,0,33,81,6,12,0,0.00,0.000000,0.000000,0.0,0.0,16,31,26,26
4,151235,consequence bhopal gas tragedy,reason behind bhopal gas tragedy,0,30,32,4,5,3,0.33,0.749981,0.599988,1.0,0.0,74,79,58,75


In [5]:
new_df.drop('Unnamed: 0', axis = 1, inplace =True)

In [6]:
new_df.shape

(49606, 17)

In [8]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
0,best marketing automation tool small mid size ...,best marketing automation tool small mid sized...
1,poor want invest,quite poor want rich
2,india live abroad met guy france party want date,e thapar university thapar university institut...
3,many people u hate southern state,boyfriend doesnt feel guilty hurt cried tellin...
4,consequence bhopal gas tragedy,reason behind bhopal gas tragedy


In [9]:
final_df = new_df.drop(columns=['question1','question2'])
print(final_df.shape)
final_df.head()

(49606, 15)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_share,cwc_min,cwc_max,last_word_eq,first_word_eq,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,1,53,54,8,8,7,0.44,0.874989,0.874989,1.0,1.0,99,98,99,99
1,0,16,20,3,4,2,0.29,0.666644,0.499988,0.0,0.0,61,73,50,72
2,0,48,90,9,11,0,0.00,0.000000,0.000000,0.0,0.0,26,40,30,30
3,0,33,81,6,12,0,0.00,0.000000,0.000000,0.0,0.0,16,31,26,26
4,0,30,32,4,5,3,0.33,0.749981,0.599988,1.0,0.0,74,79,58,75


In [12]:
import gensim
from gensim.models import Word2Vec
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

# Ensure you have the necessary NLTK packages
nltk.download('punkt')

# Replace NaN or non-string values with empty strings in question1 and question2
ques_df['question1'] = ques_df['question1'].fillna("").astype(str)
ques_df['question2'] = ques_df['question2'].fillna("").astype(str)

# Combine questions for Word2Vec training
questions = list(ques_df['question1']) + list(ques_df['question2'])
tokenized_questions = [word_tokenize(question.lower()) for question in questions]

# Train the Word2Vec model
model = Word2Vec(sentences=tokenized_questions, vector_size=100, window=5, min_count=1, sg=0)

# Define a function to get the average word vector for each question
def get_average_word_vector(sentence, model, vector_size):
    if not sentence:  # Handle empty or invalid sentences
        return np.zeros(vector_size)
    words = word_tokenize(sentence.lower())
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    if len(word_vectors) == 0:  # If no word vectors are found, return a zero vector
        return np.zeros(vector_size)
    
    return np.mean(word_vectors, axis=0)

# Get the average word vectors for each question
q1_arr = np.array([get_average_word_vector(question, model, 100) for question in ques_df['question1']])
q2_arr = np.array([get_average_word_vector(question, model, 100) for question in ques_df['question2']])

# Verify the shapes of q1_arr and q2_arr
print("q1_arr shape:", q1_arr.shape)
print("q2_arr shape:", q2_arr.shape)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<ipython-input-12-2a4c8201bff2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ques_df['question1'] = ques_df['question1'].fillna("").astype(str)
<ipython-input-12-2a4c8201bff2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ques_df['question2'] = ques_df['question2'].fillna("").astype(str)


q1_arr shape: (49606, 100)
q2_arr shape: (49606, 100)


In [13]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(49606, 200)

In [14]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head()

(49606, 215)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_share,cwc_min,cwc_max,last_word_eq,...,90,91,92,93,94,95,96,97,98,99
0,1,53,54,8,8,7,0.44,0.874989,0.874989,1.0,...,0.812994,-0.047199,0.192553,-0.063288,0.902283,0.226069,0.087011,-0.295854,0.122438,-0.132236
1,0,16,20,3,4,2,0.29,0.666644,0.499988,0.0,...,0.693783,0.236019,-0.010697,0.251330,0.895667,0.074056,0.094465,-0.288312,-0.352310,0.022238
2,0,48,90,9,11,0,0.00,0.000000,0.000000,0.0,...,0.653823,0.136772,0.760982,0.000940,1.272126,-0.030455,0.580520,-0.168829,0.509723,0.110468
3,0,33,81,6,12,0,0.00,0.000000,0.000000,0.0,...,0.250175,0.297643,-0.004345,0.258051,0.502317,0.421697,0.136965,-0.390650,-0.199890,0.130210
4,0,30,32,4,5,3,0.33,0.749981,0.599988,1.0,...,0.161569,0.081845,0.135822,0.223044,0.587892,0.017106,0.309029,-0.119430,-0.033150,0.176809


In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7914734932473292

In [17]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.7827050997782705

In [18]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Hyperparameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Test Accuracy: 0.7905664180608748


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam

# Define the neural network model
model = Sequential()

# Input layer (assumed the number of features is 10)
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

# Hidden layers
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))

# Output layer (binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6739 - loss: 0.6037 - val_accuracy: 0.7353 - val_loss: 0.4969
Epoch 2/10
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7196 - loss: 0.5148 - val_accuracy: 0.7451 - val_loss: 0.4869
Epoch 3/10
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7392 - loss: 0.4892 - val_accuracy: 0.7457 - val_loss: 0.4814
Epoch 4/10
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7408 - loss: 0.4850 - val_accuracy: 0.7514 - val_loss: 0.4714
Epoch 5/10
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7476 - loss: 0.4747 - val_accuracy: 0.7532 - val_loss: 0.4650
Epoch 6/10
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7502 - loss: 0.4722 - val_accuracy: 0.7475 - val_loss: 0.4732
Epoch 7/10
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7539 - loss: 0.4681 - val_accuracy: 0.7652 - val_loss: 0.4497
Epoch 8/10
1241/1241 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7575 - loss: 0.4652 - 

In [20]:
# Make predictions on the test set
y_pred_prob = model.predict(X_test)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


311/311 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.7632
